# 石油の配合最適化

##  対象とする問題
「**原油1**」、「**原油2**」、「**原油3**」から「**スーパー**」「**レギュラー**」「**ディーゼル**」の3種類のガソリンを作る問題

In [ ]:
import numpy as np

gas_names = ["super", "regular", "diesel"]
gas_data = np.array([[3000, 70, 10, 1], [2000, 60, 8, 2], [1000, 50, 6, 1]])

oil_names = ["crude1", "crude2", "crude3"]
oil_data = np.array([[5000, 45, 12, 0.5], [5000, 35, 6, 2], [5000, 25, 8, 3]])

nb_gas  = len(gas_names)
nb_oils = len(oil_names)
range_gas = range(nb_gas)
range_oil = range(nb_oils)
print("Number of gasoline types = {0}".format(nb_gas))
print("Number of crude types = {0}".format(nb_oils))

# global data
production_cost = 4
production_max = 14000
# each $1 spent on advertising increases demand by 10.
advert_return = 10

## データの準備

In [ ]:
from IPython.display import display
import pandas as pd
gaspd = pd.DataFrame([(gas_names[i],int(gas_data[i][0]),int(gas_data[i][1]),int(gas_data[i][2]),int(gas_data[i][3])) 
                      for i in range_gas])
oilpd = pd.DataFrame([(oil_names[i],int(oil_data[i][0]),int(oil_data[i][1]),int(oil_data[i][2]),oil_data[i][3]) 
                      for i in range_oil])
gaspd.columns = ['name','demand','price','octane','lead']
oilpd.columns= ['name','capacity','price','octane','lead']

In [ ]:
print("ガソリン情報:")
display(gaspd)

print("原油情報:")
display(oilpd)

## モデルの宣言

In [ ]:
from docplex.mp.model import Model
mdl = Model(name="oil_blending")

### 決定変数の定義

In [ ]:
# どの原油からどのガソリンを作るかの定義
# 3 x 3 の行列になる
blends = mdl.continuous_var_matrix(keys1=nb_oils, keys2=nb_gas, lb=0)

In [ ]:
# どのガソリンにどれだけ広告を打つかの定義
adverts = mdl.continuous_var_list(nb_gas, lb=0)

### 制約条件の定義

#### 需要

In [ ]:
# gas_data[g][0] にガソリンごとの需要が入っている
# 広告を打つことにより追加需要が発生することを加味する

mdl.add_constraints(mdl.sum(blends[o, g] for o in range(nb_oils)) == gas_data[g][0] + advert_return * adverts[g]
                   for g in range(nb_gas))
mdl.print_information()

#### 最大供給量

In [ ]:
# oil_data[o][0]には、原油ごとの供給量が入っている
# blends[o, g]を原油単位に足した結果は、供給量以下である必要がある

mdl.add_constraints(mdl.sum(blends[o,g] for g in range_gas) <= oil_data[o][0]
                    for o in range_oil)
mdl.print_information()

#### オクタン価と鉛含有率に関する制約

In [ ]:
# ガソリンごとにオクタン価は最小値以上の必要がある
# オクタン価は oil_data[o][2]、gas_data[g][2]に入っている

mdl.add_constraints(mdl.sum(blends[o,g]*(oil_data[o][2] - gas_data[g][2]) for o in range_oil) >= 0
                   for g in range_gas)
    
# ガソリンごとに鉛含有率は最大値以下の必要がある
# 鉛含有率はoil_data[o][3]、gas_data[g][3]に入っている

mdl.add_constraints(mdl.sum(blends[o,g]*(oil_data[o][3] - gas_data[g][3]) for o in range_oil) <= 0
                   for g in range_gas)

mdl.print_information()

#### 全体生産量に関する制約

In [ ]:
# 全体生産量は所定の最大値以下である必要がある

mdl.add_constraint(mdl.sum(blends) <= production_max)

mdl.print_information()

#### KPIの定義

In [ ]:
# KPIs

# 総広告費用
total_advert_cost = mdl.sum(adverts)
mdl.add_kpi(total_advert_cost, "Total advertising cost")

# 総原油コスト
total_oil_cost = mdl.sum(blends[o,g] * oil_data[o][1] for o in range_oil for g in range_gas)
mdl.add_kpi(total_oil_cost, "Total Oil cost")

# 総生産コスト
total_production_cost = production_cost * mdl.sum(blends)
mdl.add_kpi(total_production_cost, "Total production cost")

# 総収入
total_revenue = mdl.sum(blends[o,g] * gas_data[g][1] for g in range(nb_gas) for o in range(nb_oils))
mdl.add_kpi(total_revenue, "Total revenue")

# 最終KPI = 利益の最大化
# 利益 = 収入 - 原油コスト - 生産コスト - 広告コスト
mdl.maximize(total_revenue - total_oil_cost - total_production_cost - total_advert_cost)

### Decision Optimizerで問題を解く

In [ ]:
# Decision Optimaizerの呼出し

assert mdl.solve(), "Solve failed"
mdl.report()

##  ソリューション内容の精査

In [ ]:
#  個別KPIの取得

all_kpis = [(kp.name, kp.compute()) for kp in mdl.iter_kpis()]
kpis_bd = pd.DataFrame(all_kpis, columns=['kpi', 'value'])
display(kpis_bd)

In [ ]:
blend_values = [ [ blends[o,g].solution_value for g in range_gas] for o in range_oil]
total_gas_prods = [sum(blend_values[o][g] for o in range_oil) for g in range_gas]

prods = list(zip(gas_names, total_gas_prods))
prods_bd = pd.DataFrame(prods)

In [ ]:
# KPIのグラフ化
# 総収入、個別コスト、利益

%matplotlib inline
import matplotlib.pyplot as plt
def display_pie(pie_values, pie_labels, colors=None,title=''):
    plt.axis("equal")
    plt.pie(pie_values, labels=pie_labels, colors=colors, autopct="%1.1f%%")
    plt.title(title)
    plt.show()
                                                           
display_pie( [kpnv[1] for kpnv in all_kpis], [kpnv[0] for kpnv in all_kpis],title='KPIs: Revenue - Oil Cost - Production Cost')

#### 生産量

In [ ]:
# ガソリン別生産量

display_pie(total_gas_prods, gas_names, colors=["green", "goldenrod", "lightGreen"],title='Gasoline Total Production')

In [ ]:
# 原油・製品別生産量内訳

sblends = [(gas_names[n], oil_names[o], round(blends[o,n].solution_value)) for n in range_gas for o in range_oil]
blends_bd = pd.DataFrame(sblends)
display(blends_bd)

In [ ]:
# 製品別にグループ化してグラフ表示

f, barplot = plt.subplots(1, figsize=(16,5))

bar_width = 0.1
offset = 0.12
rho = 0.7

# position of left-bar boundaries
bar_l = [o for o in range_oil]

mbar_w = 3*bar_width+2*max(0, offset-bar_width)

tick_pos = [b*rho + mbar_w/2.0 for b in bar_l]

colors = ['olive', 'lightgreen', 'cadetblue']

for i in range_oil:
    barplot.bar([b*rho + (i*offset) for b in bar_l], 
                blend_values[i], width=bar_width, color=colors[i], label=oil_names[i])

plt.xticks(tick_pos, gas_names)
barplot.set_xlabel("gasolines")
barplot.set_ylabel("blend")
plt.legend(loc="upper right")
plt.title('Blend Repartition\n')
                                                                          

# Set a buffer around the edge
plt.xlim([0, max(tick_pos)+mbar_w +0.5])

plt.show()

In [ ]:
# 原油2からディーゼルを作っていないことの確認

print("* value of blend[crude2, diesel] is %g" % blends[1,2])